In [9]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

from utils import Ansi

SEED = 42

In [2]:
dataset1 = load_dataset('ai-forever/ru-reviews-classification').select_columns(['text']) #75k
dataset2 = load_dataset('ai-forever/georeview-classification').select_columns(['text']) #60k
dataset3 = load_dataset('d0rj/geo-reviews-dataset-2023').select_columns(['text']) #500k
dataset4 = load_dataset('blankbuffoon/wb-reviews-5mln').select_columns(['text']) #10M

ru_reviews_dataset = concatenate_datasets([
    *dataset1.values(), 
    *dataset2.values(), 
    *dataset3.values(), 
    *dataset4.values(),
])

ru_reviews_dataset

Dataset({
    features: ['text'],
    num_rows: 10635000
})

In [3]:
random_indices = np.random.choice(len(ru_reviews_dataset), 5, replace=False)
for idx in random_indices:
    print('`>>> ', ru_reviews_dataset[int(idx)]['text'])

`>>>  Очень крутой журнал для любителей дачи и цветов. Бери любой вариант, высаживай и жди красоты
`>>>  Спасибо, всё пришло целое
`>>>  Просто класс. Уже несколько стирок, но цвет и качество не изменилось.
`>>>  Мега няшные подкрадульки. Всё по размеру, дефектов не наблюдаю
`>>>  Гобелен не может быть мягким, (но он не наждачка, просто плотный); не особо понежиться можно, лежа на них, но супер объемное, праздничное настроение эти наволочки  приносят в дом! Качество - великолепное


In [6]:
def preprocess_ru_reviews(example):
    texts = example['text']
    processed_texts = []
    for text in texts:
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\d+', '[NUM]', text)
        processed_texts.append(text)
    return {'text': processed_texts}

ru_reviews_dataset = ru_reviews_dataset.map(preprocess_ru_reviews, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/10635000 [00:00<?, ? examples/s]

In [7]:
ru_reviews_dataset.set_format('pandas')
ru_reviews_dataset = ru_reviews_dataset[:]
print(f"Количество дубликатов: {Ansi.bold}{ru_reviews_dataset.duplicated().sum():,}{Ansi.end}")
ru_reviews_dataset = ru_reviews_dataset.drop_duplicates()
print(f"Итоговый размер датасета: {Ansi.bold}{ru_reviews_dataset.shape[0]:,}{Ansi.end}")

Количество дубликатов: 1,283,688
Итоговый размер датасета: 9,351,312


In [10]:
train, val = train_test_split(ru_reviews_dataset, random_state=SEED, test_size=50000)

In [11]:
train.to_csv('data/ru_reviews_train.csv', index=False)
val.to_csv('data/ru_reviews_val.csv', index=False)